<a href="https://colab.research.google.com/github/Adityajoshi526/Adityajoshi526/blob/main/Automated_finder.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install googlemaps

  Preparing metadata (setup.py) ... done
  Created wheel for googlemaps: filename=googlemaps-4.10.0-py3-none-any.whl size=40714 sha256=47f6a579dd629b8fa857cd26433f9b8173124d0dc047833060d3577a1c6b445b
  Stored in directory: /root/.cache/pip/wheels/f1/09/77/3cc2f5659cbc62341b30f806aca2b25e6a26c351daa5b1f49a
Successfully built googlemaps


In [ ]:
# Automated Competitive Store Mapping Tool - Enhanced Version
# For Google Colab - Complete Solution with API Key Auto-injection, Address Validation, and Latest Street View

import pandas as pd
import numpy as np
import json
import re
import os
import time
import requests
from bs4 import BeautifulSoup
import ipywidgets as widgets
from IPython.display import display, HTML, clear_output, Javascript
import folium
from folium.plugins import MarkerCluster, HeatMap

# Initialize notebook
print("Initializing Enhanced Automated Competitive Store Mapping Tool...")
print("This tool will help you analyze your company's locations versus competitors")
print("and identify strategic expansion hotspots specific to your industry.")
print("\nNOTE: This tool requires Google API keys. API usage will incur charges.")
print("A test mode is available to limit costs during initial testing.")

# Global variable to store the API key
STORED_API_KEY = None

# Step 1: Set up environment and get API key
def setup_environment():
    global STORED_API_KEY

    # Create API key input
    api_key = widgets.Password(
        description='Google API Key:',
        placeholder='Enter your Google API Key',
        layout=widgets.Layout(width='500px')
    )

    # Display input and message
    print("\nPlease enter your Google API Key.")
    print("This key needs Maps JavaScript API, Places API, Geocoding API, and Maps Static API enabled.")
    display(api_key)

    # Create a button to confirm
    confirm_button = widgets.Button(
        description='Confirm API Key',
        button_style='primary',
        tooltip='Click to confirm your API key'
    )

    def on_confirm_button_clicked(b):
        global gmaps, API_KEY, STORED_API_KEY
        API_KEY = api_key.value
        STORED_API_KEY = API_KEY  # Store for later use in HTML generation
        try:
            # Test the API key with a simple request
            import googlemaps
            gmaps = googlemaps.Client(key=API_KEY)
            geocode_result = gmaps.geocode("New Delhi, India")

            if geocode_result:
                clear_output()
                print("✅ API Key validated successfully!")
                print("Your API key will be automatically included in the downloaded HTML file.")
                print("Proceeding to industry selection...")
                get_industry_selection()
            else:
                print("❌ API Key validation failed. Please check your key and try again.")
        except Exception as e:
            print(f"❌ Error validating API key: {str(e)}")
            print("Make sure you have the 'googlemaps' package installed. Run: !pip install googlemaps")

    confirm_button.on_click(on_confirm_button_clicked)
    display(confirm_button)

    # Display pip install instruction
    print("\nIf you see an import error, run the following command first:")
    print("!pip install googlemaps requests beautifulsoup4 folium ipywidgets")

# Step 2: Get industry selection
def get_industry_selection():
    # Define industry types and subtypes
    industries = {
        "Retail": [
            "Fashion & Apparel",
            "Jewelry & Accessories",
            "Electronics",
            "Furniture & Home Decor",
            "Grocery & Supermarkets",
            "Luxury Goods",
            "Footwear",
            "Sports & Fitness"
        ],
        "Financial Services": [
            "Banking",
            "Insurance - Life",
            "Insurance - Health",
            "Insurance - General",
            "Wealth Management",
            "Microfinance"
        ],
        "Food & Beverage": [
            "Restaurants",
            "Coffee Shops",
            "Fast Food",
            "Bakeries",
            "Ice Cream Parlors"
        ],
        "Healthcare": [
            "Hospitals",
            "Clinics",
            "Pharmacies",
            "Diagnostic Centers",
            "Dental Clinics"
        ],
        "Automotive": [
            "Car Dealerships",
            "Service Centers",
            "Two-Wheeler Showrooms",
            "Auto Parts"
        ],
        "Real Estate": [
            "Residential Sales",
            "Commercial Leasing",
            "Property Management"
        ],
        "Education": [
            "Schools",
            "Colleges",
            "Training Centers",
            "Coaching Institutes"
        ],
        "Hospitality": [
            "Hotels",
            "Resorts",
            "Homestays"
        ]
    }

    # Create dropdown for industry selection
    industry_dropdown = widgets.Dropdown(
        options=list(industries.keys()),
        description='Industry:',
        layout=widgets.Layout(width='300px')
    )

    # Create initial subtype dropdown
    subtype_dropdown = widgets.Dropdown(
        options=industries[list(industries.keys())[0]],
        description='Sub-type:',
        layout=widgets.Layout(width='300px')
    )

    # Update subtypes when industry changes
    def on_industry_change(change):
        subtype_dropdown.options = industries[change['new']]
        subtype_dropdown.value = industries[change['new']][0]

    industry_dropdown.observe(on_industry_change, names='value')

    # Display dropdowns
    print("\nPlease select your industry and sub-type:")
    display(widgets.VBox([industry_dropdown, subtype_dropdown]))

    # Create a button to confirm
    confirm_button = widgets.Button(
        description='Confirm Industry',
        button_style='primary',
        tooltip='Click to confirm your industry selection'
    )

    def on_confirm_button_clicked(b):
        global INDUSTRY, SUBTYPE
        INDUSTRY = industry_dropdown.value
        SUBTYPE = subtype_dropdown.value

        clear_output()
        print(f"✅ Selected Industry: {INDUSTRY} - {SUBTYPE}")
        print("Proceeding to company selection...")
        get_company_information()

    confirm_button.on_click(on_confirm_button_clicked)
    display(confirm_button)

# Step 3: Get company information
def get_company_information():
    # Create text inputs for company names
    main_company = widgets.Text(
        description='Your Company:',
        placeholder='e.g., Raymond, Tanishq, HDFC Bank',
        layout=widgets.Layout(width='500px')
    )

    competitor1 = widgets.Text(
        description='Competitor 1:',
        placeholder='Enter main competitor',
        layout=widgets.Layout(width='500px')
    )

    competitor2 = widgets.Text(
        description='Competitor 2:',
        placeholder='Enter another competitor (optional)',
        layout=widgets.Layout(width='500px')
    )

    competitor3 = widgets.Text(
        description='Competitor 3:',
        placeholder='Enter another competitor (optional)',
        layout=widgets.Layout(width='500px')
    )

    # Create a test mode checkbox
    test_mode = widgets.Checkbox(
        value=True,
        description='Enable Test Mode (recommended to limit API costs)',
        layout=widgets.Layout(width='500px')
    )

    # Create state and city inputs for test mode
    state_input = widgets.Text(
        description='Test State:',
        placeholder='e.g., Maharashtra, Delhi',
        layout=widgets.Layout(width='500px')
    )

    city_input = widgets.Text(
        description='Test City:',
        placeholder='e.g., Mumbai, New Delhi',
        layout=widgets.Layout(width='500px')
    )

    # Show/hide state and city based on test mode
    def on_test_mode_change(change):
        if change['new']:
            state_box.layout.display = 'flex'
            city_box.layout.display = 'flex'
        else:
            state_box.layout.display = 'none'
            city_box.layout.display = 'none'

    test_mode.observe(on_test_mode_change, names='value')

    # Group state and city inputs
    state_box = widgets.HBox([state_input])
    city_box = widgets.HBox([city_input])

    # Display inputs
    print("\nPlease enter your company and competitor information:")
    display(widgets.VBox([main_company, competitor1, competitor2, competitor3, test_mode, state_box, city_box]))

    # Create a button to confirm
    confirm_button = widgets.Button(
        description='Start Analysis',
        button_style='primary',
        tooltip='Click to start the analysis'
    )

    def on_confirm_button_clicked(b):
        global MAIN_COMPANY, COMPETITORS, TEST_MODE, TEST_STATE, TEST_CITY

        # Validate company inputs
        if not main_company.value.strip():
            print("❌ Please enter your company name.")
            return

        if not competitor1.value.strip():
            print("❌ Please enter at least one competitor.")
            return

        # Validate test mode inputs
        if test_mode.value:
            if not state_input.value.strip() or not city_input.value.strip():
                print("❌ In test mode, both state and city are required.")
                return

        # Save settings
        MAIN_COMPANY = main_company.value.strip()

        COMPETITORS = [
            competitor1.value.strip()
        ]

        if competitor2.value.strip():
            COMPETITORS.append(competitor2.value.strip())

        if competitor3.value.strip():
            COMPETITORS.append(competitor3.value.strip())

        TEST_MODE = test_mode.value
        TEST_STATE = state_input.value.strip() if TEST_MODE else None
        TEST_CITY = city_input.value.strip() if TEST_MODE else None

        clear_output()
        print(f"✅ Analyzing {MAIN_COMPANY} vs. {', '.join(COMPETITORS)}")
        if TEST_MODE:
            print(f"📍 Test mode enabled for: {TEST_CITY}, {TEST_STATE}")

        # Start the analysis process
        start_analysis()

    confirm_button.on_click(on_confirm_button_clicked)
    display(confirm_button)

# Helper function to validate address
def validate_address(place_data, company_name, format_name):
    """
    Validates if a place is actually a store of the given company.
    Returns a validation status and confidence score.
    """
    validation_status = "Unknown"
    confidence_score = 0.0
    validation_notes = []

    # Extract place information
    name = place_data.get('name', '').lower()
    address = place_data.get('formatted_address', '').lower()
    types = place_data.get('types', [])
    business_status = place_data.get('business_status', '')

    # Check if business is operational
    if business_status == 'CLOSED_PERMANENTLY':
        return "Invalid", 0.0, ["Business permanently closed"]

    # Company name matching
    company_lower = company_name.lower()
    format_lower = format_name.lower()

    # Strong match: Company name in place name
    if company_lower in name:
        confidence_score += 0.4
        validation_notes.append("Company name matches")

    # Partial match: Key words from company name
    company_words = company_lower.split()
    matching_words = sum(1 for word in company_words if len(word) > 3 and word in name)
    if matching_words > 0:
        confidence_score += 0.2 * (matching_words / len(company_words))
        validation_notes.append(f"{matching_words} keyword(s) match")

    # Check if format name appears
    if format_lower in name and format_lower != company_lower:
        confidence_score += 0.1
        validation_notes.append("Format name matches")

    # Type-based validation
    expected_types = get_expected_place_types(INDUSTRY, SUBTYPE)
    matching_types = [t for t in types if t in expected_types]
    if matching_types:
        confidence_score += 0.2
        validation_notes.append(f"Place type matches: {', '.join(matching_types)}")

    # Address validation - check if it's in a reasonable location
    invalid_keywords = ['head office', 'corporate', 'factory', 'warehouse', 'distribution',
                       'manufacturing', 'regional office', 'zonal office', 'back office']

    for keyword in invalid_keywords:
        if keyword in name or keyword in address:
            confidence_score -= 0.3
            validation_notes.append(f"Non-retail location: {keyword}")

    # Rating check - legitimate stores usually have ratings
    if 'rating' in place_data and place_data['rating'] is not None:
        confidence_score += 0.1
        validation_notes.append("Has customer ratings")

    # Determine validation status based on confidence score
    if confidence_score >= 0.6:
        validation_status = "Valid"
    elif confidence_score >= 0.3:
        validation_status = "Uncertain"
    else:
        validation_status = "Invalid"

    return validation_status, confidence_score, validation_notes

# Helper function to get expected place types based on industry
def get_expected_place_types(industry, subtype):
    """
    Returns expected Google Places types for the given industry/subtype
    """
    type_mapping = {
        "Retail": {
            "default": ["store", "establishment", "point_of_interest", "shopping_mall"],
            "Fashion & Apparel": ["clothing_store", "store", "establishment"],
            "Jewelry & Accessories": ["jewelry_store", "store", "establishment"],
            "Electronics": ["electronics_store", "store", "establishment"],
            "Furniture & Home Decor": ["furniture_store", "home_goods_store", "store"],
            "Grocery & Supermarkets": ["supermarket", "grocery_or_supermarket", "store"],
            "Footwear": ["shoe_store", "store", "establishment"],
            "Sports & Fitness": ["store", "establishment", "gym"]
        },
        "Financial Services": {
            "default": ["bank", "finance", "establishment", "point_of_interest"],
            "Banking": ["bank", "atm", "finance"],
            "Insurance - Life": ["insurance_agency", "finance", "establishment"],
            "Insurance - Health": ["insurance_agency", "health", "finance"],
            "Insurance - General": ["insurance_agency", "finance", "establishment"]
        },
        "Food & Beverage": {
            "default": ["food", "restaurant", "establishment"],
            "Restaurants": ["restaurant", "food", "establishment"],
            "Coffee Shops": ["cafe", "food", "establishment"],
            "Fast Food": ["restaurant", "food", "meal_takeaway"],
            "Bakeries": ["bakery", "food", "store"],
            "Ice Cream Parlors": ["food", "store", "establishment"]
        },
        "Healthcare": {
            "default": ["health", "establishment", "point_of_interest"],
            "Hospitals": ["hospital", "health", "establishment"],
            "Clinics": ["doctor", "health", "establishment"],
            "Pharmacies": ["pharmacy", "health", "store"],
            "Diagnostic Centers": ["health", "establishment"],
            "Dental Clinics": ["dentist", "health", "establishment"]
        },
        "Automotive": {
            "default": ["car_dealer", "car_repair", "store", "establishment"],
            "Car Dealerships": ["car_dealer", "store", "establishment"],
            "Service Centers": ["car_repair", "establishment"],
            "Two-Wheeler Showrooms": ["store", "establishment"],
            "Auto Parts": ["store", "car_repair", "establishment"]
        },
        "Real Estate": {
            "default": ["real_estate_agency", "establishment", "point_of_interest"]
        },
        "Education": {
            "default": ["school", "university", "establishment", "point_of_interest"],
            "Schools": ["school", "primary_school", "secondary_school"],
            "Colleges": ["university", "establishment"],
            "Training Centers": ["establishment", "point_of_interest"],
            "Coaching Institutes": ["establishment", "point_of_interest"]
        },
        "Hospitality": {
            "default": ["lodging", "establishment", "point_of_interest"],
            "Hotels": ["lodging", "establishment"],
            "Resorts": ["lodging", "establishment", "spa"],
            "Homestays": ["lodging", "establishment"]
        }
    }

    # Get types for the specific industry and subtype
    if industry in type_mapping:
        if subtype in type_mapping[industry]:
            return type_mapping[industry][subtype]
        else:
            return type_mapping[industry]["default"]

    # Fallback to general types
    return ["store", "establishment", "point_of_interest"]

# Step 4: Start the analysis process
def start_analysis():
    print("\n🔍 Starting automated store location analysis with address validation...")

    # Step 4.1: Find store formats for each company
    print("\n📋 Finding store formats for all companies...")
    all_companies = [MAIN_COMPANY] + COMPETITORS
    company_formats = {}

    # Progress indicator
    progress_bar = widgets.IntProgress(
        value=0,
        min=0,
        max=len(all_companies),
        description='Finding formats:',
        bar_style='info',
        orientation='horizontal'
    )
    display(progress_bar)

    for i, company in enumerate(all_companies):
        progress_bar.value = i
        formats = find_store_formats(company)
        company_formats[company] = formats
        time.sleep(1)  # Avoid hitting rate limits

    progress_bar.value = len(all_companies)

    # Display found formats
    print("\n🏪 Found store formats:")
    for company, formats in company_formats.items():
        print(f"  {company}: {', '.join(formats)}")

    # Step 4.2: Fetch and validate store locations
    print("\n📍 Finding and validating store locations...")
    locations = {}

    # Progress bar for location fetching
    loc_progress = widgets.IntProgress(
        value=0,
        min=0,
        max=sum(len(formats) for formats in company_formats.values()),
        description='Finding locations:',
        bar_style='info',
        orientation='horizontal'
    )
    display(loc_progress)

    # Option to manually input store counts for testing
    test_mode_checkbox = widgets.Checkbox(
        value=False,
        description='Use random test data (no API calls)',
        disabled=False,
        indent=False
    )
    display(test_mode_checkbox)
    test_data_button = widgets.Button(
        description="Generate Test Data",
        button_style='info',
        tooltip='Generate random test data without API calls'
    )
    display(test_data_button)

    def generate_test_data(b):
        global locations
        import random

        # Clear previous locations
        locations = {}

        # Generate test data for each company
        for company, formats in company_formats.items():
            locations[company] = []

            # Base coordinates for test area
            if TEST_MODE:
                # Try to geocode the test city, or use Delhi as fallback
                try:
                    geocode_result = gmaps.geocode(f"{TEST_CITY}, {TEST_STATE}, India")
                    if geocode_result:
                        base_lat = geocode_result[0]['geometry']['location']['lat']
                        base_lng = geocode_result[0]['geometry']['location']['lng']
                    else:
                        base_lat, base_lng = 28.6139, 77.2090  # Delhi
                except:
                    base_lat, base_lng = 28.6139, 77.2090  # Delhi
            else:
                base_lat, base_lng = 28.6139, 77.2090  # Delhi

            # Generate random store locations for each format
            for format_name in formats:
                # Generate 5-15 random locations per format
                num_stores = random.randint(5, 15)

                for i in range(num_stores):
                    # Random offset from base coordinates (within ~10km)
                    lat_offset = random.uniform(-0.1, 0.1)
                    lng_offset = random.uniform(-0.1, 0.1)

                    # Create store data
                    store = {
                        'name': f"{format_name} #{i+1}",
                        'address': f"Test Address #{i+1}, {TEST_CITY if TEST_MODE else 'New Delhi'}",
                        'lat': base_lat + lat_offset,
                        'lng': base_lng + lng_offset,
                        'rating': round(random.uniform(3.0, 5.0), 1),
                        'company': company,
                        'format': format_name,
                        'place_id': f"test-id-{company}-{format_name}-{i}",
                        'city': TEST_CITY if TEST_MODE else 'New Delhi',
                        'state': TEST_STATE if TEST_MODE else 'Delhi',
                        'validation_status': 'Valid (Test Data)',
                        'confidence_score': 1.0,
                        'validation_notes': ['Test data - not validated']
                    }

                    locations[company].append(store)

            # Update progress
            loc_progress.value = sum(len(locations[c]) for c in locations)

        print("\n📊 Test data generated:")
        for company, locs in locations.items():
            print(f"  {company}: {len(locs)} test locations")

        # Proceed to next step
        loc_progress.value = loc_progress.max

        # Step 4.3: Define hotspot establishment types based on industry
        hotspot_types = define_hotspot_types()

        # Step 4.4: Generate the HTML map
        print("\n🗺️ Generating interactive map...")
        map_html = generate_map_html(locations, hotspot_types)

        print("\n✅ Analysis complete!")

    test_data_button.on_click(generate_test_data)

    # Continue with real data if not using test data
    progress_counter = 0
    for company, formats in company_formats.items():
        locations[company] = []

        for format_name in formats:
            search_name = f"{format_name}"

            if TEST_MODE:
                # In test mode, only search in the specified city and state
                search_query = f"{search_name} in {TEST_CITY}, {TEST_STATE}"
                store_locations = fetch_and_validate_store_locations(search_query, company, format_name)
                locations[company].extend(store_locations)
            else:
                # In full mode, search across India
                search_query = f"{search_name} in India"
                store_locations = fetch_and_validate_store_locations(search_query, company, format_name)
                locations[company].extend(store_locations)

            progress_counter += 1
            loc_progress.value = progress_counter
            time.sleep(2)  # Avoid hitting rate limits

    # Remove duplicates based on location coordinates
    for company in locations:
        unique_locations = {}
        for loc in locations[company]:
            # Use lat/lng as a unique key
            key = f"{loc['lat']:.5f}_{loc['lng']:.5f}"
            if key not in unique_locations:
                unique_locations[key] = loc

        locations[company] = list(unique_locations.values())

    # Display counts including validation status
    print("\n📊 Store counts by company (validated):")
    for company, locs in locations.items():
        valid_count = sum(1 for loc in locs if loc.get('validation_status') == 'Valid')
        uncertain_count = sum(1 for loc in locs if loc.get('validation_status') == 'Uncertain')
        invalid_count = sum(1 for loc in locs if loc.get('validation_status') == 'Invalid')

        print(f"  {company}: {len(locs)} total locations")
        print(f"    ✅ Valid: {valid_count}")
        print(f"    ⚠️  Uncertain: {uncertain_count}")
        print(f"    ❌ Invalid: {invalid_count}")

    # Step 4.3: Define hotspot establishment types based on industry
    hotspot_types = define_hotspot_types()

    # Step 4.4: Generate the HTML map with API key
    print("\n🗺️ Generating interactive map with your API key...")
    map_html = generate_map_html(locations, hotspot_types)

    print("\n✅ Analysis complete!")
    print("Your API key has been automatically included in the HTML file.")

# Helper function to find store formats for a company
def find_store_formats(company_name):
    """
    Uses Google search to find likely store format names for a company.
    """
    # Default formats if specific ones can't be found
    default_formats = [f"{company_name} Store"]

    try:
        # First, try a direct search for store types
        search_query = f"{company_name} store types India"
        search_result = gmaps.places(search_query)

        # Process search results to extract potential format names
        potential_formats = set()

        # Format detection patterns based on industry
        patterns = [
            fr"{re.escape(company_name)}\s+(Shop|Store|Outlet|Boutique|Showroom|Branch|Office|Studio|Salon)",
            fr"(Flagship|Express|Select|Premium|Luxury|Mini|Studio)\s+{re.escape(company_name)}",
            fr"{re.escape(company_name)}\s+(Exclusive|Retail|Collection|Mart|Point|Hub|Center|Lounge)"
        ]

        # Check if we have any results
        if not search_result or 'results' not in search_result or len(search_result['results']) < 3:
            # Fall back to simple format
            return default_formats

        # Extract names and apply patterns
        for place in search_result['results']:
            name = place.get('name', '')

            # Apply each pattern to detect format names
            for pattern in patterns:
                matches = re.findall(pattern, name, re.IGNORECASE)
                if matches:
                    for match in matches:
                        if isinstance(match, tuple):
                            match = match[0]  # Take first group if multiple
                        potential_format = f"{company_name} {match}"
                        potential_formats.add(potential_format)

        # Add company name itself (for places that might just be listed as the company name)
        potential_formats.add(company_name)

        # If we found formats, return them, otherwise use defaults
        formats = list(potential_formats) if potential_formats else default_formats

        # Ensure reasonable number of formats (not too many or too few)
        if len(formats) > 5:
            # Too many formats, keep most likely ones
            formats = formats[:5]
        elif len(formats) < 1:
            formats = default_formats

        return formats

    except Exception as e:
        print(f"Error finding formats for {company_name}: {str(e)}")
        return default_formats

# Enhanced helper function to fetch and validate store locations
def fetch_and_validate_store_locations(query, company, format_name):
    """
    Fetches store locations using the Places API and validates them.
    """
    locations = []

    try:
        # Search for places
        search_result = gmaps.places(query)

        # Process results
        if 'results' in search_result:
            for place in search_result['results']:
                # Extract location info
                if 'geometry' in place and 'location' in place['geometry']:
                    # Validate the address
                    validation_status, confidence_score, validation_notes = validate_address(place, company, format_name)

                    location = {
                        'name': place.get('name', format_name),
                        'address': place.get('formatted_address', 'Address unavailable'),
                        'lat': place['geometry']['location']['lat'],
                        'lng': place['geometry']['location']['lng'],
                        'rating': place.get('rating', None),
                        'company': company,
                        'format': format_name,
                        'place_id': place.get('place_id', ''),
                        'validation_status': validation_status,
                        'confidence_score': confidence_score,
                        'validation_notes': validation_notes
                    }

                    # Find city and state from address components if available
                    location['city'] = extract_locality_from_address(place)
                    location['state'] = extract_state_from_address(place)

                    # Only add if validation status is not Invalid
                    if validation_status != "Invalid":
                        locations.append(location)

        # If we got no valid results and the query wasn't specific enough, try a more specific query
        if len(locations) == 0 and 'in India' in query:
            # Try major cities
            major_cities = ["Mumbai", "Delhi", "Bangalore", "Hyderabad", "Chennai",
                           "Kolkata", "Pune", "Ahmedabad", "Jaipur"]

            for city in major_cities[:3]:  # Limit to 3 cities to avoid excessive API calls
                city_query = f"{format_name} in {city}"
                city_result = gmaps.places(city_query)

                if 'results' in city_result:
                    for place in city_result['results']:
                        if 'geometry' in place and 'location' in place['geometry']:
                            # Validate the address
                            validation_status, confidence_score, validation_notes = validate_address(place, company, format_name)

                            if validation_status != "Invalid":
                                location = {
                                    'name': place.get('name', format_name),
                                    'address': place.get('formatted_address', f'In {city}'),
                                    'lat': place['geometry']['location']['lat'],
                                    'lng': place['geometry']['location']['lng'],
                                    'rating': place.get('rating', None),
                                    'company': company,
                                    'format': format_name,
                                    'place_id': place.get('place_id', ''),
                                    'city': city,
                                    'state': '',  # We'll try to extract this later
                                    'validation_status': validation_status,
                                    'confidence_score': confidence_score,
                                    'validation_notes': validation_notes
                                }

                                locations.append(location)

                time.sleep(2)  # Avoid hitting rate limits

        # Test mode filtering - ensure we only return locations in the specified city and state
        if TEST_MODE:
            filtered_locations = []
            for loc in locations:
                if ((loc['city'] and TEST_CITY.lower() in loc['city'].lower()) or
                    (loc['address'] and TEST_CITY.lower() in loc['address'].lower())):
                    filtered_locations.append(loc)

            locations = filtered_locations

        return locations

    except Exception as e:
        print(f"Error fetching locations for {query}: {str(e)}")
        return []

# Helper function to extract city from address components
def extract_locality_from_address(place):
    """
    Extract the city/locality from a place result
    """
    if 'address_components' in place:
        for component in place['address_components']:
            if 'locality' in component['types']:
                return component['long_name']

    # If we can't find a locality in address components,
    # try to extract from formatted address
    if 'formatted_address' in place:
        # Look for common city pattern in Indian addresses (before state or PIN code)
        address = place['formatted_address']
        # Try to match city pattern - typically before PIN code
        pin_match = re.search(r'([A-Za-z\s]+)[\s,]+([\d]{6})', address)
        if pin_match:
            return pin_match.group(1).strip()

    return ""

# Helper function to extract state from address components
def extract_state_from_address(place):
    """
    Extract the state from a place result
    """
    if 'address_components' in place:
        for component in place['address_components']:
            if 'administrative_area_level_1' in component['types']:
                return component['long_name']

    return ""

# Define hotspot types based on industry
def define_hotspot_types():
    """
    Returns a list of establishment types to use for hotspot analysis based on industry
    """
    # Start with some general types all industries might be interested in
    general_types = [
        {"name": "Shopping Malls", "type": "shopping_mall", "category": "Shopping", "weight": 3.0},
        {"name": "High-end Streets", "type": "route", "keyword": "shopping street|market", "category": "Shopping", "weight": 2.5},
        {"name": "Tourist Areas", "type": "tourist_attraction", "category": "Entertainment", "weight": 2.0}
    ]

    # Industry-specific types
    industry_types = {
        "Retail": {
            "Fashion & Apparel": [
                {"name": "Department Stores", "type": "department_store", "category": "Shopping", "weight": 3.0},
                {"name": "Designer Stores", "type": "clothing_store", "keyword": "designer|luxury", "category": "Fashion", "weight": 2.5},
                {"name": "Lifestyle Stores", "type": "department_store", "keyword": "lifestyle", "category": "Shopping", "weight": 2.2},
                {"name": "Premium Malls", "type": "shopping_mall", "keyword": "premium|luxury", "category": "Shopping", "weight": 3.0}
            ],
            "Jewelry & Accessories": [
                {"name": "Jewelry Stores", "type": "jewelry_store", "category": "Shopping", "weight": 3.0},
                {"name": "Luxury Stores", "type": "clothing_store", "keyword": "luxury", "category": "Shopping", "weight": 2.5},
                {"name": "Premium Hotels", "type": "lodging", "keyword": "5 star", "category": "Accommodation", "weight": 2.0}
            ],
            "Electronics": [
                {"name": "Electronics Stores", "type": "electronics_store", "category": "Shopping", "weight": 3.0},
                {"name": "Tech Hubs", "type": "point_of_interest", "keyword": "tech hub", "category": "Business", "weight": 2.0},
                {"name": "IT Offices", "type": "point_of_interest", "keyword": "software park|IT park", "category": "Business", "weight": 1.5}
            ],
            "Furniture & Home Decor": [
                {"name": "Furniture Stores", "type": "furniture_store", "category": "Shopping", "weight": 3.0},
                {"name": "Home Improvement", "type": "home_goods_store", "category": "Shopping", "weight": 2.5},
                {"name": "Interior Design Studios", "type": "point_of_interest", "keyword": "interior design", "category": "Services", "weight": 2.0}
            ],
            "Grocery & Supermarkets": [
                {"name": "Supermarkets", "type": "supermarket", "category": "Shopping", "weight": 3.0},
                {"name": "Residential Areas", "type": "point_of_interest", "keyword": "residential society|apartment complex", "category": "Residential", "weight": 2.5}
            ]
        },
        "Financial Services": {
            "Banking": [
                {"name": "Banks", "type": "bank", "category": "Finance", "weight": 3.0},
                {"name": "Commercial Areas", "type": "point_of_interest", "keyword": "commercial complex|business district", "category": "Business", "weight": 2.5},
                {"name": "Corporate Offices", "type": "point_of_interest", "keyword": "corporate park|office complex", "category": "Business", "weight": 2.0}
            ],
            "Insurance - Life": [
                {"name": "Corporate Offices", "type": "point_of_interest", "keyword": "corporate park|office complex", "category": "Business", "weight": 3.0},
                {"name": "Hospitals", "type": "hospital", "category": "Healthcare", "weight": 2.5},
                {"name": "Premium Residential", "type": "point_of_interest", "keyword": "luxury apartment|premium housing", "category": "Residential", "weight": 2.0}
            ],
            "Insurance - Health": [
                {"name": "Hospitals", "type": "hospital", "category": "Healthcare", "weight": 3.0},
                {"name": "Clinics", "type": "doctor", "category": "Healthcare", "weight": 2.5},
                {"name": "Corporate Offices", "type": "point_of_interest", "keyword": "corporate park|office complex", "category": "Business", "weight": 2.0},
                {"name": "Pharmacies", "type": "pharmacy", "category": "Healthcare", "weight": 2.0}
            ]
        },
        "Healthcare": {
            "Hospitals": [
                {"name": "Residential Areas", "type": "point_of_interest", "keyword": "residential society|apartment complex", "category": "Residential", "weight": 3.0},
                {"name": "Corporate Offices", "type": "point_of_interest", "keyword": "corporate park|office complex", "category": "Business", "weight": 2.5},
                {"name": "Hotels", "type": "lodging", "category": "Accommodation", "weight": 2.0}
            ],
            "Pharmacies": [
                {"name": "Hospitals", "type": "hospital", "category": "Healthcare", "weight": 3.0},
                {"name": "Clinics", "type": "doctor", "category": "Healthcare", "weight": 2.5},
                {"name": "Residential Areas", "type": "point_of_interest", "keyword": "residential society|apartment complex", "category": "Residential", "weight": 2.0}
            ]
        },
        "Food & Beverage": {
            "Restaurants": [
                {"name": "Shopping Malls", "type": "shopping_mall", "category": "Shopping", "weight": 3.0},
                {"name": "Corporate Offices", "type": "point_of_interest", "keyword": "corporate park|office complex", "category": "Business", "weight": 2.5},
                {"name": "Tourist Areas", "type": "tourist_attraction", "category": "Entertainment", "weight": 2.0}
            ],
            "Coffee Shops": [
                {"name": "Corporate Offices", "type": "point_of_interest", "keyword": "corporate park|office complex", "category": "Business", "weight": 3.0},
                {"name": "Universities", "type": "university", "category": "Education", "weight": 2.5},
                {"name": "Shopping Malls", "type": "shopping_mall", "category": "Shopping", "weight": 2.0}
            ]
        }
    }

    # Get specific types for the selected industry and subtype
    specific_types = []

    if INDUSTRY in industry_types:
        if SUBTYPE in industry_types[INDUSTRY]:
            specific_types = industry_types[INDUSTRY][SUBTYPE]
        # If subtype not found, use first subtype as default
        elif industry_types[INDUSTRY]:
            first_subtype = list(industry_types[INDUSTRY].keys())[0]
            specific_types = industry_types[INDUSTRY][first_subtype]

    # Combine general and specific types
    return general_types + specific_types

# Generate the HTML map with API key automatically injected
def generate_map_html(locations, hotspot_types):
    """
    Generate HTML for the interactive map with API key automatically included
    """
    # Get all companies
    all_companies = list(locations.keys())
    main_company = all_companies[0]
    competitors = all_companies[1:]

    # Define colors for each company
    company_colors = {
        main_company: {
            'primary': '#006400',  # Dark green
            'secondary': '#228B22'  # Forest green
        }
    }

    # Competitor colors - red, blue, purple
    competitor_colors = [
        {'primary': '#8B0000', 'secondary': '#FF0000'},  # Red
        {'primary': '#00008B', 'secondary': '#0000FF'},  # Blue
        {'primary': '#4B0082', 'secondary': '#8A2BE2'}   # Purple
    ]

    for i, competitor in enumerate(competitors):
        if i < len(competitor_colors):
            company_colors[competitor] = competitor_colors[i]
        else:
            # Fallback color if more than 3 competitors
            company_colors[competitor] = {'primary': '#FF8C00', 'secondary': '#FFA500'}  # Orange

    # Prepare priority cities (if any)
    priority_cities = {}
    if TEST_MODE:
        # In test mode, just use the test city as high priority
        priority_cities = {
            'high': [{
                'city': TEST_CITY,
                'state': TEST_STATE,
                'lat': None,
                'lng': None
            }],
            'medium': [],
            'low': []
        }
    else:
        # Auto-detect priority cities based on store concentration
        priority_cities = detect_priority_cities(locations)

    # Find coordinates for priority cities if not already set
    for priority in priority_cities:
        for city in priority_cities[priority]:
            if city['lat'] is None or city['lng'] is None:
                try:
                    geocode_result = gmaps.geocode(f"{city['city']}, {city['state']}, India")
                    if geocode_result:
                        city['lat'] = geocode_result[0]['geometry']['location']['lat']
                        city['lng'] = geocode_result[0]['geometry']['location']['lng']
                except Exception as e:
                    print(f"Error geocoding priority city: {str(e)}")

    # Create HTML content with API key injected
    html_content = """
    <!DOCTYPE html>
    <html>
    <head>
        <title>Competitive Store Mapping Tool</title>
        <meta charset="utf-8">
        <meta name="viewport" content="width=device-width, initial-scale=1.0">
        <style>
            html, body {
                height: 100%;
                margin: 0;
                padding: 0;
                font-family: Arial, sans-serif;
            }
            #map {
                height: 100%;
                width: 100%;
                position: absolute;
                top: 0;
                left: 0;
                z-index: 0;
            }
            .panel {
                position: absolute;
                background: white;
                border-radius: 8px;
                box-shadow: 0 2px 6px rgba(0,0,0,0.3);
                overflow-y: auto;
                z-index: 1;
            }
            .control-panel {
                top: 10px;
                right: 10px;
                width: 300px;
                max-height: calc(100% - 20px);
                padding: 15px;
            }
            .legend {
                bottom: 30px;
                left: 10px;
                padding: 10px;
                max-height: 300px;
            }
            .hotspot-panel {
                bottom: 30px;
                right: 10px;
                width: 300px;
                max-height: 250px;
                padding: 12px;
                display: none;
            }
            h2 {
                margin-top: 0;
                margin-bottom: 15px;
                font-size: 20px;
            }
            h3 {
                margin-top: 5px;
                margin-bottom: 10px;
                font-size: 16px;
            }
            .tabs {
                display: flex;
                margin-bottom: 15px;
                border-bottom: 1px solid #ddd;
            }
            .tab {
                padding: 8px 15px;
                cursor: pointer;
                border-bottom: 3px solid transparent;
            }
            .tab.active {
                font-weight: bold;
                border-bottom: 3px solid #007bff;
                color: #007bff;
            }
            .tab-content {
                display: none;
            }
            .tab-content.active {
                display: block;
            }
            .section {
                margin-bottom: 15px;
                padding-bottom: 10px;
                border-bottom: 1px solid #eee;
            }
            .section:last-child {
                border-bottom: none;
            }
            .section-title {
                font-weight: bold;
                margin-bottom: 8px;
            }
            .color-box {
                display: inline-block;
                width: 16px;
                height: 16px;
                margin-right: 8px;
                border: 1px solid #ccc;
                vertical-align: middle;
            }
            .color-circle {
                display: inline-block;
                width: 16px;
                height: 16px;
                border-radius: 50%;
                margin-right: 8px;
                vertical-align: middle;
            }
            select {
                width: 100%;
                padding: 6px;
                margin-bottom: 10px;
                border: 1px solid #ccc;
                border-radius: 4px;
            }
            button {
                padding: 6px 12px;
                background: #007bff;
                color: white;
                border: none;
                border-radius: 4px;
                cursor: pointer;
                font-size: 14px;
            }
            button:hover {
                background: #0069d9;
            }
            button:disabled {
                background: #cccccc;
                cursor: not-allowed;
            }
            label {
                display: block;
                margin-bottom: 5px;
            }
            input[type="checkbox"] {
                margin-right: 6px;
            }
            .checkbox-list {
                max-height: 150px;
                overflow-y: auto;
                border: 1px solid #eee;
                padding: 5px;
                margin-top: 5px;
                margin-bottom: 10px;
            }
            .category-group {
                margin-bottom: 8px;
            }
            .category-name {
                font-weight: bold;
                margin-bottom: 3px;
            }
            .establishment-item {
                margin-left: 20px;
            }
            #loading {
                position: fixed;
                top: 50%;
                left: 50%;
                transform: translate(-50%, -50%);
                background: rgba(255, 255, 255, 0.9);
                padding: 20px;
                border-radius: 8px;
                box-shadow: 0 0 10px rgba(0,0,0,0.2);
                text-align: center;
                z-index: 1000;
                display: none;
            }
            #error-message {
                position: fixed;
                top: 20px;
                left: 50%;
                transform: translateX(-50%);
                background: #f8d7da;
                color: #721c24;
                padding: 10px 20px;
                border-radius: 5px;
                box-shadow: 0 0 10px rgba(0,0,0,0.2);
                z-index: 1000;
                display: none;
            }
            #street-view-container {
                display: none;
                position: fixed;
                top: 0;
                left: 0;
                width: 100%;
                height: 100%;
                background: rgba(0, 0, 0, 0.7);
                z-index: 1001;
            }
            #street-view {
                position: absolute;
                top: 50%;
                left: 50%;
                transform: translate(-50%, -50%);
                width: 90%;
                height: 90%;
                background: #fff;
                border-radius: 8px;
                overflow: hidden;
            }
            #close-street-view {
                position: absolute;
                top: 10px;
                right: 10px;
                background: #fff;
                color: #000;
                border: none;
                border-radius: 50%;
                width: 30px;
                height: 30px;
                font-size: 20px;
                line-height: 30px;
                text-align: center;
                cursor: pointer;
                z-index: 1002;
            }
            .validation-badge {
                display: inline-block;
                padding: 2px 6px;
                border-radius: 3px;
                font-size: 11px;
                margin-left: 5px;
                font-weight: bold;
            }
            .validation-valid {
                background-color: #28a745;
                color: white;
            }
            .validation-uncertain {
                background-color: #ffc107;
                color: black;
            }
            .validation-invalid {
                background-color: #dc3545;
                color: white;
            }
        </style>
    </head>
    <body>
        <div id="map"></div>

        <div class="panel control-panel">
            <h2>Competitive Analysis Tool</h2>

            <div class="tabs">
                <div class="tab active" data-tab="filters">Filters</div>
                <div class="tab" data-tab="heatmap">Heatmap</div>
                <div class="tab" data-tab="validation">Validation</div>
            </div>

            <div id="filters-tab" class="tab-content active">
                <div class="section">
                    <div class="section-title">Search Location</div>
                    <select id="state-select">
                        <option value="">Select State</option>
                    </select>
                    <select id="city-select" disabled>
                        <option value="">Select City</option>
                    </select>
                    <button id="go-location">Go to Location</button>
                </div>

                <div class="section">
                    <div class="section-title">Companies</div>
                    <!-- Company checkboxes will be populated by JavaScript -->
                    <div id="company-toggles"></div>
                </div>

                <div class="section">
                    <div class="section-title">Store Formats</div>
                    <!-- Format checkboxes will be populated by JavaScript -->
                    <div id="format-toggles"></div>
                </div>

                <div class="section">
                    <div class="section-title">Priority Cities</div>
                    <label>
                        <input type="checkbox" id="high-priority" checked> High Priority
                    </label>
                    <label>
                        <input type="checkbox" id="medium-priority" checked> Medium Priority
                    </label>
                    <label>
                        <input type="checkbox" id="low-priority" checked> Low Priority
                    </label>
                </div>

                <div class="section">
                    <button id="find-hotspots" disabled>Find Expansion Hotspots</button>
                    <button id="reset-filters" style="background: #6c757d; margin-left: 5px;">Reset</button>
                </div>
            </div>

            <div id="heatmap-tab" class="tab-content">
                <div class="section">
                    <div class="section-title">Customize Heatmap</div>
                    <p style="font-size: 12px; margin-top: 0;">Select establishment types to include in the analysis</p>

                    <label>
                        <input type="checkbox" id="select-all"> Select All Types
                    </label>

                    <div class="checkbox-list" id="establishment-list">
                        <!-- Will be populated by JavaScript -->
                    </div>

                    <label>
                        <input type="checkbox" id="show-heatmap"> Show Heatmap on Map
                    </label>

                    <div style="margin-top: 10px;">
                        <button id="update-heatmap" disabled>Update Heatmap</button>
                        <button id="clear-heatmap" disabled style="background: #6c757d; margin-left: 5px;">Clear</button>
                    </div>
                </div>
            </div>

            <div id="validation-tab" class="tab-content">
                <div class="section">
                    <div class="section-title">Address Validation Filter</div>
                    <p style="font-size: 12px; margin-top: 0;">Show stores based on validation status</p>

                    <label>
                        <input type="checkbox" id="show-valid" checked>
                        <span class="validation-badge validation-valid">Valid</span> - High confidence matches
                    </label>
                    <label>
                        <input type="checkbox" id="show-uncertain" checked>
                        <span class="validation-badge validation-uncertain">Uncertain</span> - Possible matches
                    </label>
                    <label>
                        <input type="checkbox" id="show-invalid">
                        <span class="validation-badge validation-invalid">Invalid</span> - Low confidence
                    </label>

                    <div style="margin-top: 10px;">
                        <button id="apply-validation-filter">Apply Filter</button>
                    </div>
                </div>

                <div class="section">
                    <div class="section-title">Validation Summary</div>
                    <div id="validation-summary">
                        <!-- Will be populated by JavaScript -->
                    </div>
                </div>
            </div>
        </div>

        <div class="panel legend">
            <h3>Legend</h3>

            <div class="section">
                <div class="section-title">Companies</div>
                <!-- Legend items will be populated by JavaScript -->
                <div id="company-legend"></div>
            </div>

            <div class="section">
                <div class="section-title">Priority Areas</div>
                <div><span class="color-circle" style="background-color: rgba(220, 53, 69, 0.3);"></span> High Priority</div>
                <div><span class="color-circle" style="background-color: rgba(253, 126, 20, 0.3);"></span> Medium Priority</div>
                <div><span class="color-circle" style="background-color: rgba(255, 193, 7, 0.3);"></span> Low Priority</div>
            </div>

            <div class="section">
                <div class="section-title">Hotspots</div>
                <div><span class="color-box" style="background-color: #FFD700;"></span> Expansion Hotspot</div>
            </div>
        </div>

        <div class="panel hotspot-panel" id="hotspot-panel">
            <h3>Expansion Hotspots</h3>
            <div id="hotspot-content"></div>
        </div>

        <div id="loading">
            <div style="margin-bottom: 10px;">
                <div class="spinner-border" role="status" style="width: 2rem; height: 2rem; border: 0.25em solid currentColor; border-right-color: transparent; border-radius: 50%; display: inline-block; animation: spinner-border .75s linear infinite;"></div>
            </div>
            <div id="loading-text">Loading...</div>
        </div>

        <div id="error-message"></div>

        <!-- Street View Container -->
        <div id="street-view-container">
            <button id="close-street-view">×</button>
            <div id="street-view"></div>
        </div>

        <script>
            // Store data from Python
            const allStores = LOCATIONS_PLACEHOLDER;
            const statesCities = STATES_CITIES_PLACEHOLDER;
            const priorityAreas = PRIORITY_AREAS_PLACEHOLDER;
            const establishmentTypes = ESTABLISHMENT_TYPES_PLACEHOLDER;
            const industryType = INDUSTRY_TYPE_PLACEHOLDER;
            const companyColors = COMPANY_COLORS_PLACEHOLDER;

            // Global variables
            let map;
            let infoWindow;
            let selectedLocation = null;
            let heatmap = null;
            let panorama = null;
            let streetViewService = null;
            let heatmapData = [];
            let clickListener = null;

            // Store markers
            let markers = {
                companies: {},
                hotspots: [],
                priorityCircles: {
                    'high': [],
                    'medium': [],
                    'low': []
                }
            };

            // Initialize map
            function initMap() {
                try {
                    console.log("Initializing map...");
                    showLoading("Initializing map...");

                    // Create map centered on India
                    map = new google.maps.Map(document.getElementById('map'), {
                        center: { lat: 22.5937, lng: 78.9629 },
                        zoom: 5,
                        mapTypeControl: true,
                        streetViewControl: true,
                        fullscreenControl: true
                    });

                    // Initialize Street View service
                    streetViewService = new google.maps.StreetViewService();

                    // Create info window
                    infoWindow = new google.maps.InfoWindow();

                    // Add event listener for map load
                    google.maps.event.addListenerOnce(map, 'idle', function() {
                        console.log("Map loaded successfully");

                        // Continue with initialization
                        createPriorityAreas();
                        createStoreMarkers();
                        setupCompanyFilters();
                        populateDropdowns();
                        populateEstablishmentTypes();
                        setupEventListeners();
                        updateValidationSummary();

                        hideLoading();

                        // If in test mode, zoom to test city automatically
                        if (priorityAreas.high && priorityAreas.high.length > 0) {
                            const firstCity = priorityAreas.high[0];
                            if (firstCity.lat && firstCity.lng) {
                                map.setCenter({ lat: firstCity.lat, lng: firstCity.lng });
                                map.setZoom(12);

                                // Set as selected location
                                selectedLocation = {
                                    lat: firstCity.lat,
                                    lng: firstCity.lng,
                                    city: firstCity.city,
                                    state: firstCity.state
                                };

                                // Enable buttons
                                document.getElementById('find-hotspots').disabled = false;
                                document.getElementById('update-heatmap').disabled = false;
                                document.getElementById('clear-heatmap').disabled = false;
                            }
                        }
                    });

                    // Add error listener
                    google.maps.event.addListener(map, 'error', function(e) {
                        console.error("Map error:", e);
                        showError("Error loading map. Please check your API key.");
                    });

                } catch (error) {
                    console.error("Error initializing map:", error);
                    showError("Failed to initialize map. Check console for details.");
                    hideLoading();
                }
            }

            // Create priority area circles
            function createPriorityAreas() {
                try {
                    console.log("Creating priority areas...");

                    const priorityColors = {
                        'high': {
                            fill: 'rgba(220, 53, 69, 0.3)',
                            stroke: 'rgba(220, 53, 69, 0.8)'
                        },
                        'medium': {
                            fill: 'rgba(253, 126, 20, 0.3)',
                            stroke: 'rgba(253, 126, 20, 0.8)'
                        },
                        'low': {
                            fill: 'rgba(255, 193, 7, 0.3)',
                            stroke: 'rgba(255, 193, 7, 0.8)'
                        }
                    };

                    // Create circles for each priority area
                    for (const priority in priorityAreas) {
                        const areas = priorityAreas[priority];

                        areas.forEach(area => {
                            // Skip if no lat/lng
                            if (!area.lat || !area.lng) return;

                            const circle = new google.maps.Circle({
                                strokeColor: priorityColors[priority].stroke,
                                strokeOpacity: 1,
                                strokeWeight: 1,
                                fillColor: priorityColors[priority].fill,
                                fillOpacity: 1,
                                map: map,
                                center: { lat: area.lat, lng: area.lng },
                                radius: 3000,  // 3km radius
                                zIndex: 1  // Below markers
                            });

                            // Add click listener
                            circle.addListener('click', () => {
                                const content = `
                                    <div style="max-width: 250px;">
                                        <h3>${area.city}, ${area.state}</h3>
                                        <p><strong>Priority:</strong> ${priority.charAt(0).toUpperCase() + priority.slice(1)}</p>
                                    </div>
                                `;

                                infoWindow.setContent(content);
                                infoWindow.setPosition({ lat: area.lat, lng: area.lng });
                                infoWindow.open(map);
                            });

                            // Store reference
                            markers.priorityCircles[priority].push(circle);
                        });
                    }

                    console.log("Priority areas created");
                } catch (error) {
                    console.error("Error creating priority areas:", error);
                }
            }

            // Create store markers
            function createStoreMarkers() {
                try {
                    console.log("Creating store markers...");

                    // Initialize company markers
                    for (const company in allStores) {
                        markers.companies[company] = {};
                    }

                    // Process stores for each company
                    for (const company in allStores) {
                        const stores = allStores[company];
                        const color = companyColors[company];

                        // Group stores by format
                        const formatGroups = {};
                        stores.forEach(store => {
                            const format = store.format || 'Default';
                            if (!formatGroups[format]) {
                                formatGroups[format] = [];
                            }
                            formatGroups[format].push(store);
                        });

                        // Create markers for each format
                        for (const format in formatGroups) {
                            const storesInFormat = formatGroups[format];
                            markers.companies[company][format] = [];

                            storesInFormat.forEach(store => {
                                // Skip if no lat/lng
                                if (!store.lat || !store.lng) return;

                                // Determine marker color based on validation status
                                let markerColor = color.primary;
                                let markerScale = 8;
                                let markerOpacity = 0.8;

                                if (store.validation_status === 'Uncertain') {
                                    markerOpacity = 0.6;
                                    markerScale = 7;
                                } else if (store.validation_status === 'Invalid') {
                                    markerOpacity = 0.4;
                                    markerScale = 6;
                                }

                                const marker = new google.maps.Marker({
                                    position: { lat: store.lat, lng: store.lng },
                                    map: map,
                                    title: `${store.name}`,
                                    icon: {
                                        path: google.maps.SymbolPath.CIRCLE,
                                        fillColor: markerColor,
                                        fillOpacity: markerOpacity,
                                        strokeColor: '#000000',
                                        strokeWeight: 1,
                                        scale: markerScale
                                    },
                                    zIndex: 2,  // Above priority circles
                                    // Store additional data
                                    storeData: store
                                });

                                // Add click listener
                                marker.addListener('click', () => {
                                    // Build validation badge
                                    let validationBadge = '';
                                    if (store.validation_status) {
                                        const badgeClass = `validation-${store.validation_status.toLowerCase()}`;
                                        validationBadge = `<span class="validation-badge ${badgeClass}">${store.validation_status}</span>`;
                                    }

                                    // Build priority badge if applicable
                                    let priorityBadge = '';
                                    if (store.city) {
                                        // Check if this is in a priority city
                                        let cityPriority = 'none';
                                        for (const priority in priorityAreas) {
                                            const matchingCity = priorityAreas[priority].find(
                                                area => area.city === store.city
                                            );
                                            if (matchingCity) {
                                                cityPriority = priority;
                                                break;
                                            }
                                        }

                                        if (cityPriority !== 'none') {
                                            const priorityColors = {
                                                'high': '#dc3545',
                                                'medium': '#fd7e14',
                                                'low': '#ffc107'
                                            };

                                            priorityBadge = `
                                                <span style="display: inline-block; background-color: ${priorityColors[cityPriority]};
                                                color: white; padding: 2px 6px; border-radius: 3px; font-size: 12px; margin-left: 5px;">
                                                    ${cityPriority.charAt(0).toUpperCase() + cityPriority.slice(1)}
                                                </span>
                                            `;
                                        }
                                    }

                                    // Build validation notes
                                    let validationInfo = '';
                                    if (store.validation_notes && store.validation_notes.length > 0) {
                                        validationInfo = `
                                            <p><strong>Validation Details:</strong></p>
                                            <ul style="margin: 5px 0; padding-left: 20px; font-size: 12px;">
                                                ${store.validation_notes.map(note => `<li>${note}</li>`).join('')}
                                            </ul>
                                        `;
                                    }

                                    // Content for info window
                                    const content = `
                                        <div style="max-width: 300px;">
                                            <h3>${store.name} ${validationBadge} ${priorityBadge}</h3>
                                            <p><strong>Company:</strong> ${company}</p>
                                            <p><strong>Format:</strong> ${store.format}</p>
                                            <p><strong>Location:</strong> ${store.city || 'Unknown City'}, ${store.state || 'Unknown State'}</p>
                                            <p><strong>Address:</strong> ${store.address}</p>
                                            ${store.rating ? `<p><strong>Rating:</strong> ${store.rating}/5</p>` : ''}
                                            ${store.confidence_score !== undefined ? `<p><strong>Confidence Score:</strong> ${(store.confidence_score * 100).toFixed(0)}%</p>` : ''}
                                            ${validationInfo}
                                            <p><button onclick="showEnhancedStreetView(${store.lat}, ${store.lng})" style="padding: 5px 10px; background: #007bff; color: white; border: none; border-radius: 4px; cursor: pointer;">View Street View</button></p>
                                        </div>
                                    `;

                                    infoWindow.setContent(content);
                                    infoWindow.open(map, marker);

                                    // Set as selected location
                                    selectedLocation = {
                                        lat: store.lat,
                                        lng: store.lng,
                                        city: store.city || '',
                                        state: store.state || ''
                                    };

                                    // Enable buttons
                                    document.getElementById('find-hotspots').disabled = false;
                                    document.getElementById('update-heatmap').disabled = false;
                                    document.getElementById('clear-heatmap').disabled = false;

                                    // Update dropdowns
                                    if (store.city && store.state) {
                                        updateDropdownsForLocation(store.state, store.city);
                                    }
                                });

                                // Store marker reference
                                markers.companies[company][format].push(marker);
                            });
                        }
                    }

                    console.log("Store markers created");
                } catch (error) {
                    console.error("Error creating store markers:", error);
                }
            }

            // Enhanced Street View function with latest API features
            function showEnhancedStreetView(lat, lng) {
                const streetViewContainer = document.getElementById('street-view-container');
                const streetViewDiv = document.getElementById('street-view');

                // Show loading
                showLoading("Loading Street View...");

                // Use the enhanced Street View Service
                streetViewService.getPanorama({
                    location: { lat: lat, lng: lng },
                    radius: 50,
                    source: google.maps.StreetViewSource.OUTDOOR,
                    preference: google.maps.StreetViewPreference.BEST
                }, (data, status) => {
                    if (status === google.maps.StreetViewStatus.OK) {
                        // Create enhanced panorama with latest features
                        panorama = new google.maps.StreetViewPanorama(streetViewDiv, {
                            position: data.location.latLng,
                            pov: {
                                heading: 0,
                                pitch: 0
                            },
                            zoom: 1,
                            addressControl: true,
                            addressControlOptions: {
                                position: google.maps.ControlPosition.BOTTOM_CENTER
                            },
                            linksControl: true,
                            panControl: true,
                            enableCloseButton: false,
                            fullscreenControl: true,
                            zoomControl: true,
                            motionTracking: true,
                            motionTrackingControl: true,
                            // Enable latest imagery
                            imageDateControl: true
                        });

                        // Add custom close button functionality
                        streetViewContainer.style.display = 'block';

                        // Try to get the best POV automatically
                        computeOptimalPOV(panorama, lat, lng);

                        hideLoading();
                    } else {
                        // If outdoor view not available, try default
                        streetViewService.getPanorama({
                            location: { lat: lat, lng: lng },
                            radius: 100,
                            source: google.maps.StreetViewSource.DEFAULT
                        }, (data2, status2) => {
                            if (status2 === google.maps.StreetViewStatus.OK) {
                                panorama = new google.maps.StreetViewPanorama(streetViewDiv, {
                                    position: data2.location.latLng,
                                    pov: { heading: 0, pitch: 0 },
                                    zoom: 1,
                                    addressControl: true,
                                    linksControl: true,
                                    panControl: true,
                                    fullscreenControl: true,
                                    motionTracking: true,
                                    imageDateControl: true
                                });

                                streetViewContainer.style.display = 'block';
                                computeOptimalPOV(panorama, lat, lng);
                                hideLoading();
                            } else {
                                hideLoading();
                                showError("Street View is not available at this location. The area might be newly developed or in a restricted zone.");
                            }
                        });
                    }
                });
            }

            // Compute optimal point of view for Street View
            function computeOptimalPOV(panorama, targetLat, targetLng) {
                // Get the panorama position
                const panoPos = panorama.getPosition();
                if (!panoPos) return;

                const panoLat = panoPos.lat();
                const panoLng = panoPos.lng();

                // Calculate heading from panorama to target
                const heading = google.maps.geometry.spherical.computeHeading(
                    new google.maps.LatLng(panoLat, panoLng),
                    new google.maps.LatLng(targetLat, targetLng)
                );

                // Set the POV to look at the target
                panorama.setPov({
                    heading: heading,
                    pitch: 0
                });
            }

            // Setup company filter checkboxes
            function setupCompanyFilters() {
                try {
                    console.log("Setting up company filters...");

                    // Company toggles
                    const companyToggles = document.getElementById('company-toggles');

                    // Company legend
                    const companyLegend = document.getElementById('company-legend');

                    // Create filter for each company
                    for (const company in allStores) {
                        // Create company checkbox
                        const companyLabel = document.createElement('label');
                        const companyCheckbox = document.createElement('input');
                        companyCheckbox.type = 'checkbox';
                        companyCheckbox.id = `company-${company.replace(/\s+/g, '-').toLowerCase()}`;
                        companyCheckbox.checked = true;
                        companyCheckbox.dataset.company = company;
                        companyCheckbox.addEventListener('change', updateMarkerVisibility);

                        companyLabel.appendChild(companyCheckbox);
                        companyLabel.appendChild(document.createTextNode(` ${company}`));
                        companyToggles.appendChild(companyLabel);

                        // Add company to legend
                        const primaryColorBox = document.createElement('div');
                        primaryColorBox.innerHTML = `<span class="color-box" style="background-color: ${companyColors[company].primary};"></span> ${company}`;
                        companyLegend.appendChild(primaryColorBox);

                        // Create format container
                        const formatContainer = document.createElement('div');
                        formatContainer.style.marginLeft = '20px';
                        formatContainer.id = `formats-${company.replace(/\s+/g, '-').toLowerCase()}`;

                        // Get unique formats for this company
                        const formats = Object.keys(markers.companies[company]);

                        // Create checkbox for each format
                        formats.forEach(format => {
                            const formatLabel = document.createElement('label');
                            const formatCheckbox = document.createElement('input');
                            formatCheckbox.type = 'checkbox';
                            formatCheckbox.id = `format-${company.replace(/\s+/g, '-').toLowerCase()}-${format.replace(/\s+/g, '-').toLowerCase()}`;
                            formatCheckbox.checked = true;
                            formatCheckbox.dataset.company = company;
                            formatCheckbox.dataset.format = format;
                            formatCheckbox.addEventListener('change', updateMarkerVisibility);

                            formatLabel.appendChild(formatCheckbox);
                            formatLabel.appendChild(document.createTextNode(` ${format}`));
                            formatContainer.appendChild(formatLabel);
                        });

                        // Add formats to document
                        document.getElementById('format-toggles').appendChild(formatContainer);
                    }

                    console.log("Company filters set up");
                } catch (error) {
                    console.error("Error setting up company filters:", error);
                }
            }

            // Populate state and city dropdowns
            function populateDropdowns() {
                try {
                    console.log("Populating dropdowns...");

                    const stateSelect = document.getElementById('state-select');

                    // Extract unique states and cities from all stores
                    const states = new Set();
                    const citiesByState = {};

                    for (const company in allStores) {
                        allStores[company].forEach(store => {
                            if (store.state) {
                                states.add(store.state);

                                if (!citiesByState[store.state]) {
                                    citiesByState[store.state] = new Set();
                                }

                                if (store.city) {
                                    citiesByState[store.state].add(store.city);
                                }
                            }
                        });
                    }

                    // Sort states alphabetically
                    const sortedStates = Array.from(states).sort();

                    // Add state options
                    sortedStates.forEach(state => {
                        const option = document.createElement('option');
                        option.value = state;
                        option.textContent = state;
                        stateSelect.appendChild(option);
                    });

                    // Store cities by state for dropdown updates
                    window.statesCities = {};
                    for (const state in citiesByState) {
                        window.statesCities[state] = Array.from(citiesByState[state]).sort();
                    }

                    console.log("Dropdowns populated");
                } catch (error) {
                    console.error("Error populating dropdowns:", error);
                }
            }

            // Populate establishment types for heatmap
            function populateEstablishmentTypes() {
                try {
                    console.log("Populating establishment types...");

                    const container = document.getElementById('establishment-list');

                    // Group by category
                    const categories = {};
                    establishmentTypes.forEach(type => {
                        if (!categories[type.category]) {
                            categories[type.category] = [];
                        }
                        categories[type.category].push(type);
                    });

                    // Create checkboxes for each category
                    Object.keys(categories).sort().forEach(category => {
                        const types = categories[category];

                        // Create category container
                        const categoryDiv = document.createElement('div');
                        categoryDiv.className = 'category-group';

                        // Add category header
                        const categoryHeader = document.createElement('div');
                        categoryHeader.className = 'category-name';
                        categoryHeader.textContent = category;
                        categoryDiv.appendChild(categoryHeader);

                        // Add checkboxes for each type
                        types.forEach(type => {
                            const typeDiv = document.createElement('div');
                            typeDiv.className = 'establishment-item';

                            const checkbox = document.createElement('input');
                            checkbox.type = 'checkbox';
                            checkbox.id = `type-${type.name.replace(/[^a-zA-Z0-9]/g, '_')}`;
                            checkbox.dataset.type = type.type;
                            checkbox.dataset.name = type.name;
                            checkbox.dataset.weight = type.weight;

                            if (type.keyword) {
                                checkbox.dataset.keyword = type.keyword;
                            }

                            const label = document.createElement('label');
                            label.setAttribute('for', checkbox.id);
                            label.textContent = type.name;

                            typeDiv.appendChild(checkbox);
                            typeDiv.appendChild(label);
                            categoryDiv.appendChild(typeDiv);
                        });

                        // Add to container
                        container.appendChild(categoryDiv);
                    });

                    console.log("Establishment types populated");
                } catch (error) {
                    console.error("Error populating establishment types:", error);
                }
            }

            // Update validation summary
            function updateValidationSummary() {
                try {
                    const summaryDiv = document.getElementById('validation-summary');
                    let summaryHTML = '';

                    for (const company in allStores) {
                        const stores = allStores[company];
                        const validCount = stores.filter(s => s.validation_status === 'Valid').length;
                        const uncertainCount = stores.filter(s => s.validation_status === 'Uncertain').length;
                        const invalidCount = stores.filter(s => s.validation_status === 'Invalid').length;
                        const totalCount = stores.length;

                        summaryHTML += `
                            <div style="margin-bottom: 10px;">
                                <strong>${company}</strong><br>
                                <span class="validation-badge validation-valid">Valid: ${validCount}</span>
                                <span class="validation-badge validation-uncertain">Uncertain: ${uncertainCount}</span>
                                <span class="validation-badge validation-invalid">Invalid: ${invalidCount}</span><br>
                                <span style="font-size: 12px;">Total: ${totalCount}</span>
                            </div>
                        `;
                    }

                    summaryDiv.innerHTML = summaryHTML;
                } catch (error) {
                    console.error("Error updating validation summary:", error);
                }
            }

            // Set up event listeners
            function setupEventListeners() {
                try {
                    console.log("Setting up event listeners...");

                    // Tab switching
                    document.querySelectorAll('.tab').forEach(tab => {
                        tab.addEventListener('click', function() {
                            // Update active tab
                            document.querySelectorAll('.tab').forEach(t => t.classList.remove('active'));
                            this.classList.add('active');

                            // Show corresponding content
                            const tabName = this.getAttribute('data-tab');
                            document.querySelectorAll('.tab-content').forEach(content => {
                                content.classList.remove('active');
                            });
                            document.getElementById(`${tabName}-tab`).classList.add('active');
                        });
                    });

                    // Priority toggles
                    document.getElementById('high-priority').addEventListener('change', updatePriorityVisibility);
                    document.getElementById('medium-priority').addEventListener('change', updatePriorityVisibility);
                    document.getElementById('low-priority').addEventListener('change', updatePriorityVisibility);

                    // Validation toggles
                    document.getElementById('show-valid').addEventListener('change', updateMarkerVisibility);
                    document.getElementById('show-uncertain').addEventListener('change', updateMarkerVisibility);
                    document.getElementById('show-invalid').addEventListener('change', updateMarkerVisibility);
                    document.getElementById('apply-validation-filter').addEventListener('click', updateMarkerVisibility);

                    // State dropdown
                    document.getElementById('state-select').addEventListener('change', function() {
                        updateCityDropdown(this.value);
                    });

                    // City dropdown
                    document.getElementById('city-select').addEventListener('change', function() {
                        document.getElementById('go-location').disabled = !this.value;
                    });

                    // Go to location button
                    document.getElementById('go-location').addEventListener('click', goToLocation);

                    // Find hotspots button
                    document.getElementById('find-hotspots').addEventListener('click', findHotspots);

                    // Reset filters button
                    document.getElementById('reset-filters').addEventListener('click', resetFilters);

                    // Select all establishments checkbox
                    document.getElementById('select-all').addEventListener('change', function() {
                        const checked = this.checked;
                        document.querySelectorAll('#establishment-list input[type="checkbox"]').forEach(checkbox => {
                            checkbox.checked = checked;
                        });
                    });

                    // Show heatmap toggle
                    document.getElementById('show-heatmap').addEventListener('change', toggleHeatmap);

                    // Update heatmap button
                    document.getElementById('update-heatmap').addEventListener('click', updateHeatmap);

                    // Clear heatmap button
                    document.getElementById('clear-heatmap').addEventListener('click', clearHeatmap);

                    // Close street view button
                    document.getElementById('close-street-view').addEventListener('click', closeStreetView);

                    console.log("Event listeners set up");
                } catch (error) {
                    console.error("Error setting up event listeners:", error);
                }
            }

            // Update city dropdown based on selected state
            function updateCityDropdown(state) {
                try {
                    const citySelect = document.getElementById('city-select');

                    // Clear existing options except first
                    while (citySelect.options.length > 1) {
                        citySelect.remove(1);
                    }

                    // If no state selected, disable city dropdown
                    if (!state) {
                        citySelect.disabled = true;
                        document.getElementById('go-location').disabled = true;
                        return;
                    }

                    // Add city options
                    const cities = window.statesCities[state] || [];
                    cities.forEach(city => {
                        const option = document.createElement('option');
                        option.value = city;
                        option.textContent = city;
                        citySelect.appendChild(option);
                    });

                    // Enable city dropdown
                    citySelect.disabled = false;
                } catch (error) {
                    console.error("Error updating city dropdown:", error);
                }
            }

            // Update dropdowns to match selected location
            function updateDropdownsForLocation(state, city) {
                if (!state || !city) return;

                try {
                    const stateSelect = document.getElementById('state-select');
                    const citySelect = document.getElementById('city-select');

                    // Set state
                    for (let i = 0; i < stateSelect.options.length; i++) {
                        if (stateSelect.options[i].value === state) {
                            stateSelect.selectedIndex = i;
                            break;
                        }
                    }

                    // Update city dropdown
                    updateCityDropdown(state);

                    // Set city
                    for (let i = 0; i < citySelect.options.length; i++) {
                        if (citySelect.options[i].value === city) {
                            citySelect.selectedIndex = i;
                            break;
                        }
                    }

                    // Enable go button
                    document.getElementById('go-location').disabled = false;
                } catch (error) {
                    console.error("Error updating dropdowns for location:", error);
                }
            }

            // Go to selected location
            function goToLocation() {
                try {
                    const stateSelect = document.getElementById('state-select');
                    const citySelect = document.getElementById('city-select');

                    const state = stateSelect.value;
                    const city = citySelect.value;

                    if (!state || !city) return;

                    // Find coordinates for this location
                    let coords = null;

                    // Search through all stores to find a match
                    for (const company in allStores) {
                        const matchingStore = allStores[company].find(store =>
                            store.state === state && store.city === city && store.lat && store.lng
                        );

                        if (matchingStore) {
                            coords = { lat: matchingStore.lat, lng: matchingStore.lng };
                            break;
                        }
                    }

                    // If no match in stores, check priority areas
                    if (!coords) {
                        for (const priority in priorityAreas) {
                            const matchingArea = priorityAreas[priority].find(area =>
                                area.state === state && area.city === city && area.lat && area.lng
                            );

                            if (matchingArea) {
                                coords = { lat: matchingArea.lat, lng: matchingArea.lng };
                                break;
                            }
                        }
                    }

                    if (!coords) {
                        showError("No locations found in selected city");
                        return;
                    }

                    // Zoom to location
                    map.setCenter(coords);
                    map.setZoom(13);

                    // Set as selected location
                    selectedLocation = {
                        lat: coords.lat,
                        lng: coords.lng,
                        city: city,
                        state: state
                    };

                    // Enable buttons
                    document.getElementById('find-hotspots').disabled = false;
                    document.getElementById('update-heatmap').disabled = false;
                    document.getElementById('clear-heatmap').disabled = false;
                } catch (error) {
                    console.error("Error going to location:", error);
                    showError("Failed to go to selected location");
                }
            }

            // Update marker visibility based on filters
            function updateMarkerVisibility() {
                try {
                    // Get validation filter values
                    const showValid = document.getElementById('show-valid').checked;
                    const showUncertain = document.getElementById('show-uncertain').checked;
                    const showInvalid = document.getElementById('show-invalid').checked;

                    // Get all company checkboxes
                    const companyCheckboxes = document.querySelectorAll('[data-company]');

                    // Process each checkbox
                    companyCheckboxes.forEach(checkbox => {
                        const company = checkbox.dataset.company;
                        const format = checkbox.dataset.format;

                        if (format) {
                            // This is a format checkbox
                            const companyCheckbox = document.querySelector(`[data-company="${company}"]:not([data-format])`);
                            const showCompany = companyCheckbox.checked;
                            const showFormat = checkbox.checked;

                            // Check each marker in this format
                            if (markers.companies[company] && markers.companies[company][format]) {
                                markers.companies[company][format].forEach(marker => {
                                    // Get validation status from marker's store data
                                    const validationStatus = marker.storeData ? marker.storeData.validation_status : 'Valid';

                                    // Check if should be visible based on validation
                                    let showValidation = true;
                                    if (validationStatus === 'Valid' && !showValid) showValidation = false;
                                    if (validationStatus === 'Uncertain' && !showUncertain) showValidation = false;
                                    if (validationStatus === 'Invalid' && !showInvalid) showValidation = false;

                                    // Only show if all conditions are met
                                    const visible = showCompany && showFormat && showValidation;
                                    marker.setMap(visible ? map : null);
                                });
                            }
                        } else {
                            // This is a company checkbox
                            const showCompany = checkbox.checked;

                            // If company is unchecked, hide all formats
                            if (!showCompany) {
                                for (const format in markers.companies[company]) {
                                    markers.companies[company][format].forEach(marker =>
                                        marker.setMap(null));
                                }
                            } else {
                                // If company is checked, show formats based on their checkboxes
                                for (const format in markers.companies[company]) {
                                    const formatCheckbox = document.querySelector(`[data-company="${company}"][data-format="${format}"]`);
                                    const showFormat = formatCheckbox ? formatCheckbox.checked : true;

                                    markers.companies[company][format].forEach(marker => {
                                        // Check validation status
                                        const validationStatus = marker.storeData ? marker.storeData.validation_status : 'Valid';

                                        let showValidation = true;
                                        if (validationStatus === 'Valid' && !showValid) showValidation = false;
                                        if (validationStatus === 'Uncertain' && !showUncertain) showValidation = false;
                                        if (validationStatus === 'Invalid' && !showInvalid) showValidation = false;

                                        const visible = showFormat && showValidation;
                                        marker.setMap(visible ? map : null);
                                    });
                                }
                            }
                        }
                    });
                } catch (error) {
                    console.error("Error updating marker visibility:", error);
                }
            }

            // Update priority areas visibility
            function updatePriorityVisibility() {
                try {
                    // Get filter values
                    const showHigh = document.getElementById('high-priority').checked;
                    const showMedium = document.getElementById('medium-priority').checked;
                    const showLow = document.getElementById('low-priority').checked;

                    // Update priority circles
                    markers.priorityCircles['high'].forEach(circle =>
                        circle.setMap(showHigh ? map : null));

                    markers.priorityCircles['medium'].forEach(circle =>
                        circle.setMap(showMedium ? map : null));

                    markers.priorityCircles['low'].forEach(circle =>
                        circle.setMap(showLow ? map : null));
                } catch (error) {
                    console.error("Error updating priority visibility:", error);
                }
            }

            // Reset all filters
            function resetFilters() {
                try {
                    // Reset company and format filters
                    document.querySelectorAll('[data-company]').forEach(checkbox => {
                        checkbox.checked = true;
                    });

                    // Reset priority filters
                    document.getElementById('high-priority').checked = true;
                    document.getElementById('medium-priority').checked = true;
                    document.getElementById('low-priority').checked = true;

                    // Reset validation filters
                    document.getElementById('show-valid').checked = true;
                    document.getElementById('show-uncertain').checked = true;
                    document.getElementById('show-invalid').checked = false;

                    // Reset state and city dropdowns
                    document.getElementById('state-select').selectedIndex = 0;
                    updateCityDropdown('');

                    // Reset heatmap
                    document.getElementById('show-heatmap').checked = false;
                    document.getElementById('select-all').checked = false;

                    // Uncheck all establishment types
                    document.querySelectorAll('#establishment-list input[type="checkbox"]').forEach(checkbox => {
                        checkbox.checked = false;
                    });

                    // Clear heatmap
                    clearHeatmap();

                    // Clear hotspots
                    clearHotspots();

                    // Update visibility
                    updateMarkerVisibility();
                    updatePriorityVisibility();

                    // Reset map view
                    map.setCenter({ lat: 22.5937, lng: 78.9629 });
                    map.setZoom(5);

                    // Reset selected location
                    selectedLocation = null;

                    // Disable buttons
                    document.getElementById('find-hotspots').disabled = true;
                    document.getElementById('update-heatmap').disabled = true;
                    document.getElementById('clear-heatmap').disabled = true;
                    document.getElementById('go-location').disabled = true;
                } catch (error) {
                    console.error("Error resetting filters:", error);
                }
            }

            // Find retail hotspots
            function findHotspots() {
                if (!selectedLocation) {
                    showError("Please select a location first");
                    return;
                }

                try {
                    // Clear existing hotspots
                    clearHotspots();

                    // Show loading
                    showLoading("Finding expansion hotspots...");

                    // Show hotspot panel
                    document.getElementById('hotspot-panel').style.display = 'block';
                    document.getElementById('hotspot-content').innerHTML = 'Searching for expansion hotspots...';

                    // Use Places API to find hotspots
                    const service = new google.maps.places.PlacesService(map);

                    // Get selected establishment types from the establishment list
                    const selectedTypes = [];
                    document.querySelectorAll('#establishment-list input[type="checkbox"]:checked').forEach(checkbox => {
                        selectedTypes.push({
                            type: checkbox.dataset.type,
                            name: checkbox.dataset.name,
                            keyword: checkbox.dataset.keyword
                        });
                    });

                    // If no types selected, use default types based on industry
                    if (selectedTypes.length === 0) {
                        // Use first 3 types from the establishment list
                        const allTypeCheckboxes = document.querySelectorAll('#establishment-list input[type="checkbox"]');
                        for (let i = 0; i < Math.min(3, allTypeCheckboxes.length); i++) {
                            const checkbox = allTypeCheckboxes[i];
                            selectedTypes.push({
                                type: checkbox.dataset.type,
                                name: checkbox.dataset.name,
                                keyword: checkbox.dataset.keyword
                            });
                            checkbox.checked = true;
                        }
                    }

                    // Store all results
                    const allResults = {};
                    let pendingSearches = selectedTypes.length;

                    // Search for each type
                    selectedTypes.forEach(searchType => {
                        const request = {
                            location: new google.maps.LatLng(selectedLocation.lat, selectedLocation.lng),
                            radius: 5000  // 5km
                        };

                        // Add type and keyword if specified
                        if (searchType.type) {
                            request.type = searchType.type;
                        }

                        if (searchType.keyword) {
                            request.keyword = searchType.keyword;
                        }

                        service.nearbySearch(request, (results, status) => {
                            console.log(`Search for ${searchType.name}: Status = ${status}, Results = ${results ? results.length : 0}`);

                            if (status === google.maps.places.PlacesServiceStatus.OK && results && results.length > 0) {
                                allResults[searchType.name] = results;
                            } else {
                                allResults[searchType.name] = [];
                            }

                            // Check if all searches are complete
                            pendingSearches--;
                            if (pendingSearches === 0) {
                                displayHotspots(allResults);
                                hideLoading();
                            }
                        });
                    });
                } catch (error) {
                    console.error("Error finding hotspots:", error);
                    hideLoading();
                    showError("Failed to find expansion hotspots");
                    document.getElementById('hotspot-content').innerHTML = 'Error finding expansion hotspots';
                }
            }

            // Display hotspots on map and in panel
            function displayHotspots(results) {
                try {
                    const hotspotContent = document.getElementById('hotspot-content');
                    let content = `<h4>Expansion Hotspots in ${selectedLocation.city || 'Selected Area'}</h4>`;

                    // Check if we have any results
                    const totalResults = Object.values(results).flat().length;
                    if (totalResults === 0) {
                        hotspotContent.innerHTML = `
                            <h4>Expansion Hotspots in ${selectedLocation.city || 'Selected Area'}</h4>
                            <p>No expansion hotspots found or API limit reached. Try again later or select different establishment types.</p>
                        `;
                        return;
                    }

                    // Determine which companies are present in the area
                    const companiesPresent = new Set();

                    for (const company in allStores) {
                        // Check if any store of this company is within 3km of selected location
                        const hasNearbyStore = allStores[company].some(store => {
                            if (!store.lat || !store.lng) return false;

                            // Calculate distance (rough approximation)
                            const distanceKm = calculateDistance(
                                selectedLocation.lat, selectedLocation.lng,
                                store.lat, store.lng
                            );

                            return distanceKm <= 3;
                        });

                        if (hasNearbyStore) {
                            companiesPresent.add(company);
                        }
                    }

                    // Process each type of hotspot
                    for (const [type, places] of Object.entries(results)) {
                        if (places.length === 0) continue;

                        content += `<div style="margin-top: 10px;"><strong>${type}:</strong></div>`;

                        // Sort by rating
                        places.sort((a, b) => (b.rating || 0) - (a.rating || 0));

                        // Take top 5
                        const topPlaces = places.slice(0, 5);

                        // Create markers and add to panel
                        topPlaces.forEach(place => {
                            // Add marker
                            const marker = new google.maps.Marker({
                                position: place.geometry.location,
                                map: map,
                                title: place.name,
                                icon: {
                                    url: 'https://maps.google.com/mapfiles/ms/icons/yellow-dot.png'
                                },
                                zIndex: 3  // Above store markers
                            });

                            // Get place lat and lng
                            const placeLat = place.geometry.location.lat();
                            const placeLng = place.geometry.location.lng();

                            // Create recommendation based on companies present
                            let recommendation = '';

                            if (companiesPresent.size === 0) {
                                recommendation = `This area has no existing stores from your selected companies. Consider this ${type.toLowerCase()} as a new market entry point.`;
                            } else if (companiesPresent.size === 1) {
                                const company = Array.from(companiesPresent)[0];
                                if (company === Object.keys(allStores)[0]) {
                                    // Main company is present
                                    recommendation = `Your company already has presence near this ${type.toLowerCase()}. Consider upgrading existing stores or opening a specialty format.`;
                                } else {
                                    // Competitor is present
                                    recommendation = `${company} has stores in this area. Consider entering to compete directly.`;
                                }
                            } else {
                                // Multiple companies present
                                recommendation = `Multiple competitors present near this ${type.toLowerCase()}. High-demand area worth evaluating.`;
                            }

                            // Create info window content
                            const infoContent = `
                                <div style="max-width: 250px;">
                                    <h4>${place.name}</h4>
                                    <p><strong>Type:</strong> ${type}</p>
                                    <p><strong>Rating:</strong> ${place.rating ? place.rating + '/5' : 'N/A'}
                                       (${place.user_ratings_total || 0} reviews)</p>
                                    <p><strong>Address:</strong> ${place.vicinity}</p>
                                    <p><strong>Recommendation:</strong> ${recommendation}</p>
                                    <p><button onclick="showEnhancedStreetView(${placeLat}, ${placeLng})" style="padding: 5px 10px; background: #007bff; color: white; border: none; border-radius: 4px; cursor: pointer;">View Street View</button></p>
                                </div>
                            `;

                            // Add click listener
                            marker.addListener('click', () => {
                                infoWindow.setContent(infoContent);
                                infoWindow.open(map, marker);
                            });

                            // Store marker
                            markers.hotspots.push(marker);

                            // Add to panel
                            content += `
                                <div style="margin-bottom: 10px; padding-bottom: 5px; border-bottom: 1px solid #eee;">
                                    <div><strong>${place.name}</strong></div>
                                    <div>Rating: ${place.rating ? place.rating + '/5' : 'N/A'}</div>
                                    <div>${place.vicinity}</div>
                                    <div style="margin-top: 3px; font-size: 12px; font-style: italic;">${recommendation}</div>
                                    <div><button onclick="showEnhancedStreetView(${placeLat}, ${placeLng})" style="padding: 3px 6px; background: #007bff; color: white; border: none; border-radius: 3px; cursor: pointer; font-size: 12px; margin-top: 5px;">Street View</button></div>
                                </div>
                            `;
                        });
                    }

                    // Update panel
                    hotspotContent.innerHTML = content;
                } catch (error) {
                    console.error("Error displaying hotspots:", error);
                    document.getElementById('hotspot-content').innerHTML = 'Error displaying expansion hotspots';
                }
            }

            // Calculate distance between two coordinates in kilometers
            function calculateDistance(lat1, lon1, lat2, lon2) {
                const R = 6371; // Radius of the earth in km
                const dLat = deg2rad(lat2 - lat1);
                const dLon = deg2rad(lon2 - lon1);
                const a =
                    Math.sin(dLat/2) * Math.sin(dLat/2) +
                    Math.cos(deg2rad(lat1)) * Math.cos(deg2rad(lat2)) *
                    Math.sin(dLon/2) * Math.sin(dLon/2);
                const c = 2 * Math.atan2(Math.sqrt(a), Math.sqrt(1-a));
                const d = R * c; // Distance in km
                return d;
            }

            function deg2rad(deg) {
                return deg * (Math.PI/180);
            }

            // Clear hotspot markers
            function clearHotspots() {
                try {
                    markers.hotspots.forEach(marker => marker.setMap(null));
                    markers.hotspots = [];
                    document.getElementById('hotspot-panel').style.display = 'none';
                } catch (error) {
                    console.error("Error clearing hotspots:", error);
                }
            }

            // Toggle heatmap display
            function toggleHeatmap() {
                try {
                    const showHeatmap = document.getElementById('show-heatmap').checked;

                    if (showHeatmap) {
                        if (!heatmap || !heatmap.getData() || heatmap.getData().length === 0) {
                            if (!selectedLocation) {
                                showError("Please select a location first");
                                document.getElementById('show-heatmap').checked = false;
                                return;
                            }
                            updateHeatmap();
                        } else {
                            heatmap.setMap(map);
                        }

                        // Add click listener for heatmap if not already added
                        if (!clickListener) {
                            clickListener = map.addListener('click', function(e) {
                                // Only handle clicks when heatmap is visible
                                if (!heatmap || !heatmap.getMap()) return;

                                const clickLat = e.latLng.lat();
                                const clickLng = e.latLng.lng();

                                // Check if click is near any heatmap point
                                for (const point of heatmapData) {
                                    const pointLat = point.location.lat();
                                    const pointLng = point.location.lng();

                                    // Calculate distance in degrees (simplified)
                                    const distance = Math.sqrt(
                                        Math.pow(clickLat - pointLat, 2) +
                                        Math.pow(clickLng - pointLng, 2)
                                    );

                                    // If click is within threshold (approximately 100-200 meters)
                                    if (distance < 0.002) {
                                        showEnhancedStreetView(pointLat, pointLng);
                                        return;
                                    }
                                }

                                // If clicked on heatmap but not close to any specific point
                                // Use the clicked location
                                showEnhancedStreetView(clickLat, clickLng);
                            });
                        }
                    } else if (heatmap) {
                        heatmap.setMap(null);

                        // Remove click listener when heatmap is hidden
                        if (clickListener) {
                            google.maps.event.removeListener(clickListener);
                            clickListener = null;
                        }
                    }
                } catch (error) {
                    console.error("Error toggling heatmap:", error);
                }
            }

            // Update heatmap with custom establishment types
            function updateHeatmap() {
                if (!selectedLocation) {
                    showError("Please select a location first");
                    return;
                }

                try {
                    // Get selected types
                    const selectedTypes = [];
                    document.querySelectorAll('#establishment-list input[type="checkbox"]:checked').forEach(checkbox => {
                        selectedTypes.push({
                            type: checkbox.dataset.type,
                            name: checkbox.dataset.name,
                            weight: parseFloat(checkbox.dataset.weight) || 1.0,
                            keyword: checkbox.dataset.keyword
                        });
                    });

                    if (selectedTypes.length === 0) {
                        showError("Please select at least one establishment type");
                        return;
                    }

                    // Show loading
                    showLoading(`Building heatmap with ${selectedTypes.length} selected types...`);

                    // Clear existing heatmap
                    if (heatmap) {
                        heatmap.setMap(null);
                    }

                    // Turn on heatmap display
                    document.getElementById('show-heatmap').checked = true;

                    // Use Places API to find selected establishment types
                    const service = new google.maps.places.PlacesService(map);

                    // To store heatmap data
                    heatmapData = [];

                    // Track pending searches
                    let pendingSearches = selectedTypes.length;

                    // Search for each type
                    selectedTypes.forEach(searchType => {
                        const request = {
                            location: new google.maps.LatLng(selectedLocation.lat, selectedLocation.lng),
                            radius: 5000  // 5km
                        };

                        // Set type and keyword if specified
                        if (searchType.type) {
                            request.type = searchType.type;
                        }

                        if (searchType.keyword) {
                            request.keyword = searchType.keyword;
                        }

                        service.nearbySearch(request, (results, status) => {
                            console.log(`Search for ${searchType.name}: Status = ${status}, Results = ${results ? results.length : 0}`);

                            if (status === google.maps.places.PlacesServiceStatus.OK && results && results.length > 0) {
                                // Add to heatmap data
                                results.forEach(place => {
                                    heatmapData.push({
                                        location: place.geometry.location,
                                        weight: searchType.weight * (place.rating ? place.rating/5 : 0.7)
                                    });
                                });
                            }

                            // Check if all searches are complete
                            pendingSearches--;
                            if (pendingSearches === 0) {
                                // Create the heatmap
                                if (heatmapData.length > 0) {
                                    if (heatmap) {
                                        heatmap.setMap(null);
                                    }

                                    heatmap = new google.maps.visualization.HeatmapLayer({
                                        data: heatmapData,
                                        radius: 30,
                                        opacity: 0.6
                                    });

                                    heatmap.setMap(map);

                                    // Add click listener for heatmap
                                    if (!clickListener) {
                                        clickListener = map.addListener('click', function(e) {
                                            // Only handle clicks when heatmap is visible
                                            if (!heatmap || !heatmap.getMap()) return;

                                            const clickLat = e.latLng.lat();
                                            const clickLng = e.latLng.lng();

                                            // Check if click is near any heatmap point
                                            for (const point of heatmapData) {
                                                const pointLat = point.location.lat();
                                                const pointLng = point.location.lng();

                                                // Calculate distance in degrees (simplified)
                                                const distance = Math.sqrt(
                                                    Math.pow(clickLat - pointLat, 2) +
                                                    Math.pow(clickLng - pointLng, 2)
                                                );

                                                // If click is within threshold (approximately 100-200 meters)
                                                if (distance < 0.002) {
                                                    showEnhancedStreetView(pointLat, pointLng);
                                                    return;
                                                }
                                            }

                                            // If clicked on heatmap but not close to any specific point
                                            // Use the clicked location
                                            showEnhancedStreetView(clickLat, clickLng);
                                        });
                                    }
                                } else {
                                    showError("No establishments found for selected types");
                                }

                                hideLoading();
                            }
                        });
                    });
                } catch (error) {
                    console.error("Error updating heatmap:", error);
                    hideLoading();
                    showError("Failed to create heatmap");
                }
            }

            // Clear heatmap
            function clearHeatmap() {
                try {
                    if (heatmap) {
                        heatmap.setMap(null);
                        heatmap = null;
                    }

                    // Clear heatmap data
                    heatmapData = [];

                    // Remove click listener
                    if (clickListener) {
                        google.maps.event.removeListener(clickListener);
                        clickListener = null;
                    }

                    document.getElementById('show-heatmap').checked = false;
                } catch (error) {
                    console.error("Error clearing heatmap:", error);
                }
            }

            // Close Street View
            function closeStreetView() {
                document.getElementById('street-view-container').style.display = 'none';
                if (panorama) {
                    panorama = null;
                }
            }

            // Show loading indicator
            function showLoading(message) {
                try {
                    const loading = document.getElementById('loading');
                    document.getElementById('loading-text').textContent = message || 'Loading...';
                    loading.style.display = 'block';
                } catch (error) {
                    console.error("Error showing loading:", error);
                }
            }

            // Hide loading indicator
            function hideLoading() {
                try {
                    document.getElementById('loading').style.display = 'none';
                } catch (error) {
                    console.error("Error hiding loading:", error);
                }
            }

            // Show error message
            function showError(message) {
                try {
                    const errorDiv = document.getElementById('error-message');
                    errorDiv.textContent = message;
                    errorDiv.style.display = 'block';

                    // Auto-hide after 5 seconds
                    setTimeout(() => {
                        errorDiv.style.display = 'none';
                    }, 5000);
                } catch (error) {
                    console.error("Error showing error message:", error);
                }
            }

            // Log initialization
            console.log("Script loaded, waiting for Google Maps API...");
        </script>

        <script src="https://maps.googleapis.com/maps/api/js?key=API_KEY_PLACEHOLDER&libraries=places,visualization,geometry&callback=initMap" async defer></script>
    </body>
    </html>
    """

    # Replace the API key placeholder with the actual API key
    html_content = html_content.replace('API_KEY_PLACEHOLDER', STORED_API_KEY)

    # Prepare locations_data to replace placeholder
    locations_data = {}
    for company, stores in locations.items():
        # Include all locations with their validation status
        locations_data[company] = stores

    # Create a simplified structure for states and cities
    states_cities = {}

    # Extract states and cities from locations
    for company, stores in locations.items():
        for store in stores:
            if store['state'] and store['city']:
                if store['state'] not in states_cities:
                    states_cities[store['state']] = set()
                states_cities[store['state']].add(store['city'])

    # Convert sets to sorted lists
    states_cities_dict = {}
    for state, cities in states_cities.items():
        states_cities_dict[state] = sorted(list(cities))

    # Replace placeholders in HTML
    html_content = html_content.replace('LOCATIONS_PLACEHOLDER', json.dumps(locations_data))
    html_content = html_content.replace('STATES_CITIES_PLACEHOLDER', json.dumps(states_cities_dict))
    html_content = html_content.replace('PRIORITY_AREAS_PLACEHOLDER', json.dumps(priority_cities))
    html_content = html_content.replace('ESTABLISHMENT_TYPES_PLACEHOLDER', json.dumps(hotspot_types))
    html_content = html_content.replace('INDUSTRY_TYPE_PLACEHOLDER', json.dumps(f"{INDUSTRY} - {SUBTYPE}"))
    html_content = html_content.replace('COMPANY_COLORS_PLACEHOLDER', json.dumps(company_colors))

    # Process stores for each company
    for company, stores in locations.items():
        # Save each company's data to a separate CSV file
        stores_df = pd.DataFrame(stores)
        stores_df.to_csv(f"{company.replace(' ', '_')}_stores.csv", index=False)

    # Save the HTML file
    with open('competitive_store_map.html', 'w', encoding='utf-8') as f:
        f.write(html_content)

    # Provide download instructions
    print("\n✅ Map HTML file has been created as 'competitive_store_map.html'")
    print("✅ Your API key has been automatically included in the file!")
    print("✅ CSV data for each company has also been saved for your reference.")
    print("\nTo download the files in Colab:")
    print("from google.colab import files")
    print("files.download('competitive_store_map.html')")
    print("# Repeat for each company's CSV:")
    for company in locations.keys():
        print(f"files.download('{company.replace(' ', '_')}_stores.csv')")

    print("\n🎉 You can now open the HTML file directly in your browser - no need to add the API key manually!")

    return html_content

# Helper function to detect priority cities
def detect_priority_cities(locations):
    """
    Auto-detect priority cities based on store concentration.
    High priority: Cities with stores from main company and all competitors
    Medium priority: Cities with main company and at least one competitor
    Low priority: Cities with only main company or only competitors
    """
    all_companies = list(locations.keys())
    main_company = all_companies[0]
    competitors = all_companies[1:]

    # Group stores by city
    cities = {}

    for company, stores in locations.items():
        for store in stores:
            # Only consider valid or uncertain stores for priority detection
            if store.get('validation_status') in ['Valid', 'Uncertain', 'Valid (Test Data)']:
                if store['city'] and store['state'] and store['lat'] and store['lng']:
                    city_key = f"{store['city']}, {store['state']}"

                    if city_key not in cities:
                        cities[city_key] = {
                            'city': store['city'],
                            'state': store['state'],
                            'lat': store['lat'],
                            'lng': store['lng'],
                            'companies': set()
                        }

                    cities[city_key]['companies'].add(company)

    # Categorize cities by priority
    priority_cities = {
        'high': [],
        'medium': [],
        'low': []
    }

    for city_key, city_data in cities.items():
        companies_present = city_data['companies']

        if main_company in companies_present and all(comp in companies_present for comp in competitors):
            # High priority: Main company and all competitors present
            priority_cities['high'].append({
                'city': city_data['city'],
                'state': city_data['state'],
                'lat': city_data['lat'],
                'lng': city_data['lng']
            })
        elif main_company in companies_present and any(comp in companies_present for comp in competitors):
            # Medium priority: Main company and at least one competitor
            priority_cities['medium'].append({
                'city': city_data['city'],
                'state': city_data['state'],
                'lat': city_data['lat'],
                'lng': city_data['lng']
            })
        else:
            # Low priority: Only main company or only competitors
            priority_cities['low'].append({
                'city': city_data['city'],
                'state': city_data['state'],
                'lat': city_data['lat'],
                'lng': city_data['lng']
            })

    # Limit to top cities in each category if too many
    for priority in priority_cities:
        if len(priority_cities[priority]) > 10:
            priority_cities[priority] = priority_cities[priority][:10]

    return priority_cities

# Start the process
if __name__ == "__main__":
    setup_environment()

Initializing Enhanced Automated Competitive Store Mapping Tool...
This tool will help you analyze your company's locations versus competitors
and identify strategic expansion hotspots specific to your industry.

NOTE: This tool requires Google API keys. API usage will incur charges.
A test mode is available to limit costs during initial testing.

Please enter your Google API Key.
This key needs Maps JavaScript API, Places API, Geocoding API, and Maps Static API enabled.


Password(description='Google API Key:', layout=Layout(width='500px'), placeholder='Enter your Google API Key')

Button(button_style='primary', description='Confirm API Key', style=ButtonStyle(), tooltip='Click to confirm y…


If you see an import error, run the following command first:
!pip install googlemaps requests beautifulsoup4 folium ipywidgets
